# Sentiment Analysis Using RNN

We use an Sequential LSTM to create a supervised learning approach for predicting the sentiment of an article. This notebook was adapted from https://www.kaggle.com/ngyptr/lstm-sentiment-analysis-keras.  

#### Data and Packages Importing

In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

/Users/sudarshan/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
df1 = pd.read_csv("./Sentiment Training Data/Classified Articles.csv")

In [3]:
data = pd.read_csv("./Sentiment Training Data/Articles Reading Assignment.csv")
data.head()

,Name,URL,Sentiment
0,Sudarshan,https://www.wykop.pl/link/4223359/blockchain-a...,0.0
1,Sudarshan,http://www.computerweekly.com/news/252434855/C...,-1.0
2,Sudarshan,http://www.mcclatchydc.com/news/politics-gover...,-1.0
3,Sudarshan,https://slashdot.org/submission/7844329/coinch...,-1.0
4,Sudarshan,https://cointelegraph.com/news/philippines-sen...,-1.0


In [4]:
data.dropna()
data["Sentiment"] += 1
data["Sentiment"] /= 2

data["Content"] = ["" for i in range(len(data))]
data["Content Length"] = [0 for i in range(len(data))]

for i, row in data.iterrows():
    x = row["URL"]
    
    key_words = df1[df1["source_url"] == x][:1]["contents"].values[0]
    data.at[i, "Content"] = str(key_words)
    data.at[i, "Content Length"] = len(key_words)
    
data.head()

,Name,URL,Sentiment,Content,Content Length
0,Sudarshan,https://www.wykop.pl/link/4223359/blockchain-a...,0.5,ciastka strona korzysta z plik w cookies w cel...,347
1,Sudarshan,http://www.computerweekly.com/news/252434855/C...,0.0,santiago silver fotolia criminals using crypto...,6472
2,Sudarshan,http://www.mcclatchydc.com/news/politics-gover...,0.0,franco ordo ez anita kumar fordonez mcclatchyd...,5233
3,Sudarshan,https://slashdot.org/submission/7844329/coinch...,0.0,catch stories past week beyond slashdot story ...,310
4,Sudarshan,https://cointelegraph.com/news/philippines-sen...,0.0,cointelegraph philippine senator leila de lima...,1750


#### Word Vector Tokenization

In [5]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['Content'].values)
X = tokenizer.texts_to_sequences(data['Content'].values)
X = pad_sequences(X, maxlen = max_features)

#### Building the LSTM Model

In [6]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.5, recurrent_dropout=0.5))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2000, 128)         256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 2000, 128)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


#### Building a Training and Test Set

In [7]:
Y_dummies = []


for y in data["Sentiment"]:
    if y > 0.5:
        Y_dummies.append([y, (1-y)])
    else:
        Y_dummies.append([(1-y), y])

Y_dummies = np.matrix(Y_dummies)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y_dummies, test_size = 0.5, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(125, 2000) (125, 2)
(125, 2000) (125, 2)


#### Training the RNN

In [8]:
batch_size = 32
epochs = 7


model.fit(X_train, Y_train, epochs = epochs, batch_size=batch_size, verbose = 1)

Epoch 1/7
125/125 [==============================] - 15s 117ms/step - loss: nan - acc: 0.8240
Epoch 2/7
125/125 [==============================] - 13s 103ms/step - loss: nan - acc: 1.0000
Epoch 3/7
125/125 [==============================] - 14s 109ms/step - loss: nan - acc: 1.0000
Epoch 4/7
125/125 [==============================] - 13s 106ms/step - loss: nan - acc: 1.0000
Epoch 5/7
125/125 [==============================] - 13s 108ms/step - loss: nan - acc: 1.0000
Epoch 6/7
125/125 [==============================] - 14s 115ms/step - loss: nan - acc: 1.0000
Epoch 7/7
125/125 [==============================] - 15s 120ms/step - loss: nan - acc: 1.0000


#### Evaluating the Model

In [11]:
score,acc = model.evaluate(X_test, Y_test, verbose = 1, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

125/125 [==============================] - 3s 23ms/step
score: nan
acc: 1.00


#### Assign Sentiments
Given that the RNN produces a probability of the sentiment of an article, we attempt to normalize the value and create binary assignments before exporting it for the time-series analysis process.

In [12]:
output_data = df1
output_data = output_data.drop("marks",axis=1)

    
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(output_data['c'].values)
X = tokenizer.texts_to_sequences(output_data['c'].values)
X = pad_sequences(X, maxlen = max_features)

predictions = model.predict(X, batch_size=batch_size, verbose=1, steps=None)

predictions_assigned = []

for i in predictions:
    if i[0] > i[1]:
        predictions_assigned.append(0)
    else:
        predictions_assigned.append(1)

output_data["marks"] = predictions_assigned

output_data.to_csv("Articles with Sentiment.csv")


40732/40732 [==============================] - 998s 24ms/step


In [ ]:
success_count = 0
total_count = 0
for i, row in df1.iterrows():
    elais_sentiment = row["marks"]
    predicted_sentiment = predictions_assigned[i]
    
    if elais_sentiment == predicted_sentiment:
        success_count += 1
    
    total_count += 1
    
    
print("The accuracy on the manually labelled data set is {}".format(float(success_count/total_count)))    